In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
import numpy
pd.set_option('display.max_rows', 5000)
from matplotlib.pyplot import figure
%matplotlib inline
spark.sql('use db_crs_analysis_pii_t')
#Shane Zhang, Jun, 2021
#UDF for sequence analysis, find missing count, nonmissing count, max gap, max continous, last continous, last_pmt
#miss_nb number of month that is None
#miss_nb2 similar to miss_nb, but ignore leading missing months
#nonmiss_nb number of months with a non null value
#max_gap maximum number of continous month with null value
#max_gap2 similar to max_gap, but ignore leading missing months
#max_cont maximum number of continous months with non null value
#last_cont from the last month counting reversely, number of continous months with non null value
#last_pmt last non null value in the sequence
seq=[None, None, None, 3, 4, 5, None, None, 8, 9, None, 11, 12, None, 21]
from typing import List
def seq_analysis (seq)-> List[float]:
    max_gap=0.0
    max_gap2=0.0
    gap=0.0 #temp variable.
    max_cont=0.0
    cont=0
    #temp variable
    last_cont=0.0
    miss_nb=0.0
    miss_nb2=0.0
    nonmiss_nb=0.0
    
    first_nonmiss=False
    
    for x in seq:
        if x==None:
            miss_nb=miss_nb+1
            if first_nonmiss==True:
                miss_nb2=miss_nb2+1
            cont=0
            last_cont=0.0
            gap=gap+1
            if gap>max_gap:
                max_gap-gap
                
            if gap>max_gap2:
                max_gap2=gap
        else:    
            if first_nonmiss==False:
                first_nonmiss=True
                max_gap2=0.0
            nonmiss_nb=nonmiss_nb+1
            last_pmt=x
            gap=0.0
            cont=cont+1
            last_cont=last_cont+1
            if cont>max_cont:
                max_cont=max_cont+1
    return miss_nb, miss_nb2, nonmiss_nb, max_gap, max_gap2, max_cont, last_cont, last_pmt



In [ ]:
print(seq_analysis (seq))


In [ ]:
#create a spark dataframe to test
from pyspark.sql.types import *
data = [("01234567", [None, 2.0, None, None, 5.0, None, 7.0, 8.0,9.0, None, 11.0,12.0])]
schema StructType([
StructField('eci', StringType(), True),
StructField('sequence', ArrayType (DoubleType(), True), True)
])
df = spark.createDataFrame (data-data, schema=schema)
df.printSchema ()
df.show(truncate=False)




In [ ]:
#call sequence udf from spark dataframe
from pyspark.sql.functions import udf, expr
seq_udf = udf(seq_analysis, ArrayType(DoubleType()))
df2=df.select("*", seq_udf("sequence").alias("seq"))
df2=df2.selectExpr("*","seq[0] as max_gap", "seq[1] as max_cont", "seq[2] as last_cont","seq[3] as last_pmt")
df2.toPandas()